<a href="https://colab.research.google.com/github/andreytgu1990/test/blob/master/first_steps_in_BI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

pd.set_option('display.max_columns', 10)
pd.set_option('display.expand_frame_repr', False)

path_dataset = '/content/drive/MyDrive/datasets_for_analysis/e_commerce_data.zip'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv(path_dataset, encoding = 'unicode_escape')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [ ]:
# предварительная обработка датасета
def func_main(path_dataset: str):
    # считываем датасет
    df = pd.read_csv(path_dataset, encoding = 'unicode_escape', 
                     usecols = ['InvoiceDate', 'Country', 'StockCode', 'Description', 'Quantity', 'UnitPrice'],
                     parse_dates = ['InvoiceDate'])
    
    # приводим названия столбцов датасета к нижнему регистру
    df.rename(columns = str.lower, inplace = True)

    # избавляемся от времени
    df['invoicedate'] = df['invoicedate'].dt.date

    # рассчитываем сумму покупки по каждому товару
    df['amount'] = df['quantity'] * df['unitprice']

    # задаем порядок вывода столбцов для визуального контроля результата
    df_result = df[['invoicedate', 'country', 'stockcode', 'description', 'amount']]
    return df_result

In [ ]:
# обработанный датасет
data = func_main(path_dataset)
data.head()

,invoicedate,country,stockcode,description,amount
0,2010-12-01,United Kingdom,85123A,WHITE HANGING HEART T-LIGHT HOLDER,15.30
1,2010-12-01,United Kingdom,71053,WHITE METAL LANTERN,20.34
2,2010-12-01,United Kingdom,84406B,CREAM CUPID HEARTS COAT HANGER,22.00
3,2010-12-01,United Kingdom,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,20.34
4,2010-12-01,United Kingdom,84029E,RED WOOLLY HOTTIE WHITE HEART.,20.34


In [ ]:
# таблица Страны
df_country = pd.DataFrame(sorted(pd.unique(data['country'])), columns=['country'])
df_country.head()

,country
0,Australia
1,Austria
2,Bahrain
3,Belgium
4,Brazil


In [ ]:
# Таблица Товары
df_product = data[['stockcode','description']].drop_duplicates(subset = ['stockcode'], keep ='first').reset_index(drop = True)
df_product.head()

,stockcode,description
0,85123A,WHITE HANGING HEART T-LIGHT HOLDER
1,71053,WHITE METAL LANTERN
2,84406B,CREAM CUPID HEARTS COAT HANGER
3,84029G,KNITTED UNION FLAG HOT WATER BOTTLE
4,84029E,RED WOOLLY HOTTIE WHITE HEART.


In [ ]:
#  таблица продажи 
df_sale = data.groupby(['invoicedate', 'country', 'stockcode']).aggregate({'amount':'sum'}).reset_index()
df_sale.head()

,invoicedate,country,stockcode,amount
0,2010-12-01,Australia,21622,39.6
1,2010-12-01,Australia,21791,15.0
2,2010-12-01,Australia,22191,17.0
3,2010-12-01,Australia,22192,17.0
4,2010-12-01,Australia,22193,17.0


# Создаю базу с данными

In [ ]:
import sqlite3
con = sqlite3.connect('sale.db')
cur = con.cursor()

In [ ]:
# Таблица Страны                                                          # создние таблицы
cur.executescript('''DROP TABLE IF EXISTS country;
                     CREATE TABLE IF NOT EXISTS country (
                     country_id INTEGER PRIMARY KEY AUTOINCREMENT,
                     country TEXT NOT NULL UNIQUE);
                  ''')

df_country.to_sql('country', con, index=False, if_exists='append')        # запись датафрейма в таблицу

In [ ]:
# Таблица Товары                                                          # создние таблицы
cur.executescript('''DROP TABLE IF EXISTS product;                       
                     CREATE TABLE IF NOT EXISTS product (
                     product_id INTEGER PRIMARY KEY AUTOINCREMENT,
                     stockcode TEXT NOT NULL UNIQUE,
                     description TEXT);
                  ''')

df_product.to_sql('product', con, index = False, if_exists='append')      # запись датафрейма в таблицу

In [ ]:
# Таблица Продажи (временная)
cur.executescript('''DROP TABLE IF EXISTS sale_data_lake;
                     CREATE TABLE IF NOT EXISTS sale_data_lake (
                     sale_id INTEGER PRIMARY KEY AUTOINCREMENT,
                     invoicedate TEXT NOT NULL,
                     country TEXT NOT NULL,
                     stockcode TEXT NOT NULL,
                     amount REAL NOT NULL);''')

df_sale.to_sql('sale_data_lake', con, index=False, if_exists='append')    # запись датафрейма в таблицу

In [ ]:
# Таблица Продажи (основная)                                              # создние таблицы
cur.executescript('''DROP TABLE IF EXISTS sale;                           
                     CREATE TABLE IF NOT EXISTS sale (
                     sale_id INTEGER PRIMARY KEY AUTOINCREMENT,
                     invoicedate TEXT NOT NULL,
                     country_id INTEGER NOT NULL,
                     product_id INTEGER NOT NULL,
                     amount REAL NOT NULL,
                     FOREIGN KEY(country_id)  REFERENCES country(country_id),
                     FOREIGN KEY(product_id)  REFERENCES product(product_id));
                  ''')

In [ ]:
# Перегружаем данные из вспомогательной таблицы (sale_data_lake) в основную (sale)
cur.executescript('''INSERT INTO sale (invoicedate, country_id, product_id, amount)
                     SELECT  sdl.invoicedate, c.country_id, pr.product_id, sdl.amount
                     FROM sale_data_lake as sdl 
                        LEFT JOIN country as c                                       
                            ON sdl.country = c.country
 						                    LEFT JOIN product as pr 
                                    ON sdl.stockcode = pr.stockcode
                     ''')                                                 # запись результата запроса в таблицу

In [ ]:
cur.close()
con.close()

In [ ]:
# Очищаем вспомогательную таблицу
cur.executescript('''DELETE FROM sale_data_lake''')

In [ ]:
def SELECT(sql):                                                          # функция вызова запроса
  return pd.read_sql(sql, con)

In [ ]:
#  тестовый запрос SQL для прверки результата
sql = '''SELECT s.invoicedate,
                c.country,
                pr.stockcode,
                round(s.amount, 1) AS amount,
                pr.description
          FROM sale AS s 
              LEFT JOIN country AS c
                  ON s.country_id = c.country_id
                      LEFT JOIN product as pr
                          ON s.product_id = pr.product_id
        '''

SELECT(sql)


,invoicedate,country,stockcode,amount,description
0,2010-12-01,Australia,21622,39.6,VINTAGE UNION JACK CUSHION COVER
1,2010-12-01,Australia,21791,15.0,VINTAGE HEADS AND TAILS CARD GAME
2,2010-12-01,Australia,22191,17.0,IVORY DINER WALL CLOCK
3,2010-12-01,Australia,22192,17.0,BLUE DINER WALL CLOCK
4,2010-12-01,Australia,22193,17.0,RED DINER WALL CLOCK
...,...,...,...,...,...
310010,2011-12-09,United Kingdom,90206C,5.0,CRYSTAL DIAMANTE STAR BROOCH
310011,2011-12-09,United Kingdom,90208,2.5,PAIR OF PINK FLOWER CLUSTER SLIDE
310012,2011-12-09,United Kingdom,90210A,2.9,GREY ACRYLIC FACETED BANGLE
310013,2011-12-09,United Kingdom,DOT,2647.3,DOTCOM POSTAGE


# POWER BI

 Скрипт на Python 

In [ ]:
import pandas as pd
import sqlite3

#con = sqlite3.connect('C:/Users/sale.db')
con = sqlite3.connect('sale.db')
cur = con.cursor()

def select(sql):
  return pd.read_sql(sql,con)

sql = '''SELECT *
        from (SELECT s.invoicedate,
                      c.country,
                      pr.stockcode,
                      replace(round(s.amount,1), '.', ',') AS amount,
                      pr.description
               FROM sale AS s 
               LEFT JOIN country AS c 
                    ON s.country_id = c.country_id
                        LEFT JOIN product as pr
                            ON s.product_id = pr.product_id)
        '''

table = select(sql)
print(table)

       invoicedate         country stockcode  amount                         description
0       2010-12-01       Australia     21622    39,6    VINTAGE UNION JACK CUSHION COVER
1       2010-12-01       Australia     21791    15,0  VINTAGE HEADS AND TAILS CARD GAME 
2       2010-12-01       Australia     22191    17,0              IVORY DINER WALL CLOCK
3       2010-12-01       Australia     22192    17,0               BLUE DINER WALL CLOCK
4       2010-12-01       Australia     22193    17,0                RED DINER WALL CLOCK
...            ...             ...       ...     ...                                 ...
310010  2011-12-09  United Kingdom    90206C     5,0        CRYSTAL DIAMANTE STAR BROOCH
310011  2011-12-09  United Kingdom     90208     2,5   PAIR OF PINK FLOWER CLUSTER SLIDE
310012  2011-12-09  United Kingdom    90210A     2,9         GREY ACRYLIC FACETED BANGLE
310013  2011-12-09  United Kingdom       DOT  2647,3                      DOTCOM POSTAGE
310014  2011-12-09  U

После загрузки данных в систему и проверки корректности распознанных форматов можно приступать к непосредственному построению дашборда.